## Stochastic Vairationl Inference for GPRN

This notebook illustrates implementation of GPRN (Wislon, 2011) with SVI inference for Jura dataset. Swiss Jura dataset consists of geo-spatial measurements of metals Ni Cd and Zn in the Swiss region of Jura. This can be modeled using mutli-response regression methods.

The dataset in the original format can be found here :https://sites.google.com/site/goovaertspierre/pierregoovaertswebsite/download/jura-data

** Module and data imports **

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import logging
import time
import sys
from scipy.cluster.vq import kmeans
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from numpy.random import RandomState
rng = RandomState(1231)
%matplotlib inline


float_type = tf.float64
jitter_level = 1e-5
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# define model chekpoint path & tensorboard path 
modelPath = "gprn.ckpt"
tbPath    = "gprn/"

In [2]:
# main module imports
from onofftf.main import Param, DataSet, GaussKL, KernSE, GPConditional
from onofftf.utils import modelmanager
from gpflow import transforms

In [3]:
# define trian and test splits
traindf = pd.read_csv("data/train.csv")
testdf  = pd.read_csv("data/test.csv")

Xtrain = traindf[['x', 'y']].as_matrix()
Ytrain = traindf[['cd','ni','zn']].as_matrix()

Xtest = testdf[['x', 'y']].as_matrix()
Ytest = testdf[['cd','ni','zn']].as_matrix()

In [4]:
# DataSet object gives easy access for mini-batches over epochs
train_data = DataSet(Xtrain,Ytrain)

** Set modeling parameters**

In [5]:
###############################################

num_iter      = 50000           # optimization iterations
num_minibatch = 220             # mini-batch size
num_output    = Ytrain.shape[1] # number of responses
num_latent    = 2               # number of latent functions 
num_data      = Xtrain.shape[0] # number of observations 



num_inducing_f = [10            # number of inducing points for latent function \f
                  for i in range(num_latent)]

num_inducing_w = [[10           # number of inducing points for weight matrix components W
                   for i in range(num_output)]
                   for j in range(num_latent)]

qf_diag = True  # diagnoal assumption for inducing posterior S_f
qw_diag = True  # diagnoal assumption for inducing posterior S_w

kern_param_learning_rate = 1e-2
indp_param_learning_rate = 1e-2
noise_param_learning_rate = 1e-3
###############################################

** Parameter initialization**

In [6]:
init_x = lambda num_init: kmeans(Xtrain,num_init)[0]

# initialize noise parameter
init_noisevar = [0.01, 0.01,0.01]

# initialize f's
init_fkvar = np.ones((num_latent,1))*1.
init_fkell = np.ones((num_latent,1))*1.

init_Zf = [init_x(i)
           for i in num_inducing_f]

init_u_fm = [np.random.randn(num_inducing_f[i]).reshape(-1,1)*1e-2
             for i in range(num_latent)]
if qf_diag:
    init_u_fs_sqrt = [np.ones(num_inducing_f[i]).reshape(1,-1).T
                      for i in range(num_latent)]
else:
    init_u_fs_sqrt = [np.eye(num_inducing_f[i])
                      for i in range(num_latent)]


    
# initalize W's
init_wkvar = np.ones((num_latent,num_output))*1.
init_wkell = np.ones((num_latent,num_output))*1.

init_Zw = [[init_x(num_inducing_w[j][i])
            for i in range(num_output)]
            for j in range(num_latent)]

init_u_wm = [[np.random.randn(num_inducing_w[j][i]).reshape(-1,1)*1e-2
              for i in range(num_output)]
              for j in range(num_latent)]
if qw_diag:
    init_u_ws_sqrt = [[np.ones(num_inducing_w[j][i]).reshape(1,-1).T
                       for i in range(num_output)]
                       for j in range(num_latent)]
else:
    init_u_ws_sqrt = [[np.eye(num_inducing_w[j][i])
                       for i in range(num_output)]
                       for j in range(num_latent)]

** Define parameters in TensorFlow **

In [7]:
# placeholders for dataset
X = tf.placeholder(dtype = float_type)
Y = tf.placeholder(dtype = float_type)


# kernel for latent functions : K_f
with tf.name_scope("f_kern"):
    fkell = [Param(init_fkell[i],transform=transforms.Log1pe(),
                   name="lengthscale_"+str(i),learning_rate = kern_param_learning_rate,summ=True)
             for i in range(num_latent)]

    fkvar = [Param(init_fkvar[i],transform=transforms.Log1pe(),
                   name="variance_"+str(i),learning_rate = kern_param_learning_rate,summ=True)
             for i in range(num_latent)]

fkern_list = [KernSE(fkell[i],fkvar[i])
              for i in range(num_latent)]

# kernel for weight matrix functions : K_w
with tf.name_scope("w_kern"):
    wkell = [[Param(init_wkell[j][i],transform=transforms.Log1pe(),
                   name="lengthscale_"+str(j)+"_"+str(i),learning_rate = kern_param_learning_rate,summ=True)
             for i in range(num_output)]
             for j in range(num_latent)]

    wkvar = [[Param(init_wkvar[j][i],transform=transforms.Log1pe(),
                   name="variance_"+str(j)+"_"+str(i),learning_rate = kern_param_learning_rate,summ=True)
             for i in range(num_output)]
             for j in range(num_latent)]

wkern_list = [[KernSE(wkell[j][i],wkvar[j][i])
             for i in range(num_output)]
             for j in range(num_latent)]

# noise variance parameters
with tf.name_scope("likelihood"):
    noisevar_list = [Param(init_noisevar[i],transform=transforms.Log1pe(),
                     name="variance"+str(i),learning_rate = noise_param_learning_rate,summ=True)
                for i in range(num_output)]

# inducing f
with tf.name_scope("f_ind"):
    Zf_list = [Param(init_Zf[i],name="z_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
               for i in range(num_latent)]
        
    u_fm_list = [Param(init_u_fm[i],name="value_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                 for i in range(num_latent)]
    
    if qf_diag:
        u_fs_sqrt_list = [Param(init_u_fs_sqrt[i],transforms.positive,
                                name="variance_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                          for i in range(num_latent)]
    else:
        u_fs_sqrt_list = [Param(init_u_fs_sqrt[i],transforms.LowerTriangular(init_u_fs_sqrt[i].shape[0]),
                                name="variance_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                          for i in range(num_latent)]

# inducing W  
with tf.name_scope("w_ind"):
    Zw_list = [[Param(init_Zw[j][i],name="z_"+str(j)+"_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                for i in range(num_output)]
                for j in range(num_latent)]
    
    u_wm_list = [[Param(init_u_wm[j][i],name="value_"+str(j)+"_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                 for i in range(num_output)]
                 for j in range(num_latent)]    
    
    if qw_diag:
        u_ws_sqrt_list = [[Param(init_u_ws_sqrt[j][i],transforms.positive,
                                name="variance_"+str(j)+"_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                          for i in range(num_output)]
                          for j in range(num_latent)]
    else:
        u_ws_sqrt_list = [[Param(init_u_ws_sqrt[j][i],transforms.LowerTriangular(init_u_ws_sqrt[j][i].shape[0]),
                                name="variance_"+str(j)+"_"+str(i),learning_rate = indp_param_learning_rate,summ=True)
                          for i in range(num_output)]
                          for j in range(num_latent)]


**Define modeling functions**

In [8]:
# KL divergence between all inducing priors and posteriors
def build_prior_kl(u_fm_list,u_fs_sqrt_list,fkern_list,Zf_list,
                   u_wm_list,u_ws_sqrt_list,wkern_list,Zw_list):
    
    kl_f = [GaussKL(u_fm_list[i].get_tfv(),
                    u_fs_sqrt_list[i].get_tfv(),
                    fkern_list[i].K(Zf_list[i].get_tfv()))
            for i in range(num_latent)]

    kl_w = [[GaussKL(u_wm_list[j][i].get_tfv(),
                    u_ws_sqrt_list[j][i].get_tfv(),
                    wkern_list[j][i].K(Zw_list[j][i].get_tfv()))
            for i in range(num_output)]
            for j in range(num_latent)]

    kl = tf.add_n(kl_f) + tf.reduce_sum(tf.add_n(kl_w))
    
    return kl


# Build predictive posterior
def build_predict(Xnew,u_fm_list,u_fs_sqrt_list,fkern_list,Zf_list,
                       u_wm_list,u_ws_sqrt_list,wkern_list,Zw_list):
    
    # Get conditionals for f 
    fmean_list = [[] for i in range(num_latent)]
    fvar_list  = [[] for i in range(num_latent)]
    for i in range(num_latent):
        fmean_list[i], fvar_list[i] = GPConditional(Xnew,
                                                    Zf_list[i].get_tfv(),
                                                    fkern_list[i],
                                                    u_fm_list[i].get_tfv(),
                                                    full_cov=False,
                                                    q_sqrt=u_fs_sqrt_list[i].get_tfv(),
                                                    whiten=False)
    fmean = tf.concat(fmean_list,axis=1)
    fvar = tf.concat(fvar_list,axis=1)
    
    # get conditional for w
    wmean_list = [[[] for i in range(num_output)]
                      for j in range(num_latent)]
    
    wvar_list  = [[[] for i in range(num_output)]
                      for j in range(num_latent)]
    for i in range(num_output):
        for j in range(num_latent):
            wmean_list[j][i], wvar_list[j][i] = GPConditional(Xnew,
                                                              Zw_list[j][i].get_tfv(),
                                                              wkern_list[j][i],
                                                              u_wm_list[j][i].get_tfv(),
                                                              full_cov=False,
                                                              q_sqrt=u_ws_sqrt_list[j][i].get_tfv(),
                                                              whiten=False)
    wmean = tf.concat(wmean_list,axis=2)
    wvar = tf.concat(wvar_list,axis=2)

    # compute augmented f : wf : w^T f
    wfmean = tf.transpose(tf.reduce_sum(tf.multiply(tf.expand_dims(fmean,axis=0),wmean),axis=2))

    return wfmean, fmean, fvar, wmean, wvar


# variational expectations of likelihood
def variational_expectations(Y, wfmean, fmean, fvar, wmean, wvar, noisevar_list):
    N = tf.cast(tf.shape(Y)[0], float_type)
    noisevar = tf.stack([var.get_tfv() for var in noisevar_list],axis=0)

    t1 = tf.reduce_sum(tf.square(tf.subtract(Y,wfmean)),axis=0)
    t2 = tf.reduce_sum(tf.transpose(tf.reduce_sum(tf.multiply(tf.expand_dims(fvar,axis=0),tf.square(wmean)),axis=2)),axis=0)
    t3 = tf.reduce_sum(tf.transpose(tf.reduce_sum(tf.multiply(tf.expand_dims(tf.square(fmean),axis=0),wvar),axis=2)),axis=0)
    t4 = tf.reduce_sum(tf.transpose(tf.reduce_sum(tf.multiply(tf.expand_dims(fvar,axis=0),wvar),axis=2)),axis=0)

    tsum = tf.divide(t1 + t2 + t3 + t4,noisevar)

    return - 0.5 * N * tf.reduce_sum(tf.log(2 * np.pi * noisevar)) \
           - tf.reduce_sum(0.5 * tsum)

** Build lowerbound**

In [9]:
# build prior
with tf.name_scope("kl"):
    kl = build_prior_kl(u_fm_list,u_fs_sqrt_list,fkern_list,Zf_list,
                        u_wm_list,u_ws_sqrt_list,wkern_list,Zw_list)
    tf.summary.scalar('kl', kl)

# build predict function
with tf.name_scope("model_build"):
    wfmean, fmean, fvar, wmean, wvar = build_predict(X,u_fm_list,u_fs_sqrt_list,fkern_list,Zf_list,
                                                     u_wm_list,u_ws_sqrt_list,wkern_list,Zw_list)
    tf.summary.histogram('wfmean',wfmean)
    tf.summary.histogram('fmean',fmean)
    tf.summary.histogram('fvar',fvar)
    tf.summary.histogram('wmean',wmean)
    tf.summary.histogram('wvar',wvar)

# compute likelihood
with tf.name_scope("var_exp"):
    var_exp = variational_expectations(Y, wfmean, fmean, fvar, wmean, wvar, noisevar_list)
    tf.summary.scalar('var_exp', var_exp)

    # mini-batch scaling
    scale =  tf.cast(num_data, float_type) / tf.cast(num_minibatch, float_type)
    var_exp_scaled = var_exp * scale
    tf.summary.scalar('var_exp_scaled', var_exp_scaled)

# final lower bound
with tf.name_scope("cost"):
    cost =  -(var_exp_scaled - kl)
    tf.summary.scalar('cost',cost)
    


** Run model optimization op**

In [10]:
all_var_list = tf.trainable_variables()
all_lr_list = [var._learning_rate for var in all_var_list]

train_opt_group = []

for group_learning_rate in set(all_lr_list):
    _ind_bool = np.where(np.isin(np.array(all_lr_list),group_learning_rate))[0]
    group_var_list = [all_var_list[ind] for ind in _ind_bool]
    group_tf_optimizer = tf.train.AdamOptimizer(learning_rate = group_learning_rate)
    group_grad_list = tf.gradients(cost,group_var_list)
    group_grads_and_vars = list(zip(group_grad_list,group_var_list))
    
    
    group_train_op = group_tf_optimizer.apply_gradients(group_grads_and_vars)
    
    # Summarize all gradients
    for grad, var in group_grads_and_vars:
        tf.summary.histogram(var.name + '/gradient', grad)
    
    train_opt_group.append({'names':[var.name for var in group_var_list],
                            'vars':group_var_list,
                            'learning_rate':group_learning_rate,
                            'grads':group_grad_list,
                            'train_op':group_train_op})
    
train_op = tf.group(*[group['train_op'] for group in train_opt_group])  

INFO:tensorflow:Summary name f_kern/lengthscale_0:0/gradient is illegal; using f_kern/lengthscale_0_0/gradient instead.
INFO:tensorflow:Summary name f_kern/lengthscale_1:0/gradient is illegal; using f_kern/lengthscale_1_0/gradient instead.
INFO:tensorflow:Summary name f_kern/variance_0:0/gradient is illegal; using f_kern/variance_0_0/gradient instead.
INFO:tensorflow:Summary name f_kern/variance_1:0/gradient is illegal; using f_kern/variance_1_0/gradient instead.
INFO:tensorflow:Summary name w_kern/lengthscale_0_0:0/gradient is illegal; using w_kern/lengthscale_0_0_0/gradient instead.
INFO:tensorflow:Summary name w_kern/lengthscale_0_1:0/gradient is illegal; using w_kern/lengthscale_0_1_0/gradient instead.
INFO:tensorflow:Summary name w_kern/lengthscale_0_2:0/gradient is illegal; using w_kern/lengthscale_0_2_0/gradient instead.
INFO:tensorflow:Summary name w_kern/lengthscale_1_0:0/gradient is illegal; using w_kern/lengthscale_1_0_0/gradient instead.
INFO:tensorflow:Summary name w_kern/

In [ ]:
sess = tf.InteractiveSession()

# model saver
saver = tf.train.Saver()

# tensorboard summary
summ_merged = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(tbPath,
                                        graph=sess.graph)
  
sess.run(tf.global_variables_initializer())

In [ ]:
print('{:>16s}'.format("iteration"),'{:>16s}'.format("objective"),'{:>16s}'.format("var_exp"),'{:>16s}'.format("kl"))

for i in range(num_iter):
    batch = train_data.next_batch(num_minibatch)
    try:    
        summary,_ = sess.run([summ_merged,train_op],feed_dict={X : batch[0],Y : batch[1]})
        
        if i % 1000 == 0:
            _cost    = cost.eval({X : batch[0],Y : batch[1]})
            _var_exp = var_exp.eval({X : batch[0],Y :batch[1]})
            _kl      = kl.eval({X : batch[0],Y : batch[1]})
            print('{:>16d}'.format(i),'{:>16.3f}'.format(_cost),'{:>16.3f}'.format(_var_exp),'{:>16.3f}'.format(_kl))
            
            if i > 200:
                summary_writer.add_summary(summary,i)
                summary_writer.flush()

       
    except KeyboardInterrupt as e:
        print("Stopping training")
        break
        
modelmngr = modelmanager(saver, sess, modelPath)
modelmngr.save()
summary_writer.close()

       iteration        objective          var_exp               kl
               0     89626549.689    -76130619.101           48.112
            1000      4197773.730     -3526928.382        45617.135
            2000      3114582.870     -2610188.533        41679.097
            3000      1966232.773     -1639055.669        36617.235
            4000      1445957.425     -1201887.271        31008.320
            5000      1018696.645      -843924.129        25167.784
            6000       752837.374      -622227.147        20306.324
            7000       454190.262      -371785.048        16497.865
            8000       288976.306      -234505.816        12899.004
            9000       187240.044      -150585.047         9960.375
           10000       134413.608      -107469.723         7892.434
           11000       109370.042       -87511.394         6345.265
           12000        77195.701       -61194.670         5152.885
           13000        60229.166       -47468.2

** Predictions on trainset**

In [ ]:
# predictions
pred_trn_wfmean = wfmean.eval({X:Xtrain})
pred_trn_fmean = fmean.eval({X:Xtrain})
pred_trn_fvar = fvar.eval({X:Xtrain})
pred_trn_wmean = wmean.eval({X:Xtrain})
pred_trn_wvar =wvar.eval({X:Xtrain})

** Grid plot of the model fit **

In [ ]:
mpl.rcParams['figure.figsize'] = (12,6)

xplot, yplot = np.meshgrid(np.linspace(0.1,6.1,30),np.linspace(0.1,6.1,30))
Xplot = np.concatenate([xplot.reshape(-1,1),yplot.reshape(-1,1)],axis=1)

# predictions
pred_plt_wfmean = wfmean.eval({X:Xplot})
pred_plt_fmean = fmean.eval({X:Xplot})
pred_plt_fvar = fvar.eval({X:Xplot})
pred_plt_wmean = wmean.eval({X:Xplot})
pred_plt_wvar =wvar.eval({X:Xplot})

# inducing locations and values
ind_plot_f = [Zf_list[i].get_tfv().eval() 
              for i in range(num_latent)]
ind_plot_w = [[Zw_list[j][i].get_tfv().eval()
              for i in range(num_output)]
              for j in range(num_latent)]

fig = plt.figure()

# plot latent functions
gs00 = gridspec.GridSpec(num_latent, 1)
for lf in np.arange(num_latent):
    ax   = fig.add_subplot(gs00[lf,0], aspect='equal')
    cax  = ax.contourf(xplot, yplot, pred_plt_fmean[:,lf].reshape(xplot.shape),alpha=0.7,cmap='RdYlBu')
    cbar = fig.colorbar(cax,extend='max',fraction=0.046, pad=0.04)
    ax.scatter(ind_plot_f[lf][:,0],ind_plot_f[lf][:,1],s=10)
    ax.set_xlim(0.1,6.1)
    ax.set_ylim(0.1,6.1)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    if lf == 0:
        ax.set_title("latent functions")
gs00.tight_layout(fig, rect=[0, 0, 0.2, 1])

# plot weight functions
gs01 = gridspec.GridSpec(num_output,num_latent)
for lf in np.arange(num_latent):
    for wf in np.arange(num_output):
        ax   = fig.add_subplot(gs01[wf,lf],adjustable='box-forced')
        cax  = ax.contourf(xplot, yplot,pred_plt_wmean[wf,:,lf].reshape(xplot.shape),alpha=0.7,cmap='RdYlBu')
        cbar = fig.colorbar(cax,extend='max',fraction=0.046, pad=0.04)
        ax.scatter(ind_plot_w[lf][wf][:,0],ind_plot_w[lf][wf][:,1],s=10)
        ax.set_xlim(0.1,6.1)
        ax.set_ylim(0.1,6.1)
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        if lf == 0 and wf == 0:
            ax.set_title("weight functions")
gs01.tight_layout(fig, rect=[0.2, 0, 0.6,1],h_pad=0.5)

# plot output functions
gs02 = gridspec.GridSpec(num_output, 1)
for wf in np.arange(num_output):
    ax   = fig.add_subplot(gs02[wf,0], aspect='equal',adjustable='box-forced')
    cax  = ax.contourf(xplot, yplot,pred_plt_wfmean[:,wf].reshape(xplot.shape),alpha=0.7)
    cbar = fig.colorbar(cax,extend='max',fraction=0.046, pad=0.04)
    ax.set_xlim(0.1,6.1)
    ax.set_ylim(0.1,6.1)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    if wf == 0:
        ax.set_title("predictions")


gs02.tight_layout(fig, rect=[0.6, 0, 0.8, 1],h_pad=0.5)

# plot data functions
gs03 = gridspec.GridSpec(num_output, 1)
for wf in np.arange(num_output):
    ax   = fig.add_subplot(gs03[wf,0], aspect='equal',adjustable='box-forced')
    cax  = ax.scatter(Xtrain[:,0],Xtrain[:,1],c= Ytrain[:,wf],alpha=0.7,s=10)
    cbar = fig.colorbar(cax,extend='max',fraction=0.046, pad=0.04)
    ax.set_xlim(0.1,6.1)
    ax.set_ylim(0.1,6.1)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    if wf == 0:
        ax.set_title("actual")

gs03.tight_layout(fig, rect=[0.8, 0, 1, 1],h_pad=0.5)


plt.show()


**Train set predictions**

In [ ]:
mpl.rcParams['figure.figsize'] = (12,6)
fig, ax = plt.subplots(2,3,sharex='all', sharey='all')

names = ["Cd","Ni","Zn"]
for i in range(3):
    cnorm = mpl.colors.Normalize(vmin=0.,vmax=Ytrain[:,i].max())
    
    cx1 = ax[0,i].scatter(Xtrain[:,0],Xtrain[:,1],c=Ytrain[:,i]) #,norm=cnorm
    cbar1 = fig.colorbar(cx1,ax=ax[0,i],fraction=0.046, pad=0.04)
    ax[0,i].set_title("actutal - "+ names[i])
    ax[0,i].xaxis.set_ticks([])
    ax[0,i].yaxis.set_ticks([])

    cx2 = ax[1,i].scatter(Xtrain[:,0],Xtrain[:,1],c=pred_trn_wfmean[:,i]) #,norm=cnorm
    cbar2 = fig.colorbar(cx2,ax=ax[1,i],fraction=0.046, pad=0.04)
    ax[1,i].set_title("predicted - "+ names[i]+"  noise var:"+str(np.round(noisevar_list[i].get_tfv().eval(),3)))
    ax[1,i].xaxis.set_ticks([])
    ax[1,i].yaxis.set_ticks([])

fig.tight_layout()
plt.show()

In [ ]:
print("train mse:",np.sqrt(np.mean((pred_trn_wfmean - Ytrain)**2,axis=0)))
print("train mae:",np.mean(np.abs(pred_trn_wfmean - Ytrain),axis=0))

** Test set predictions **

In [ ]:
mpl.rcParams['figure.figsize'] = (12,6)
fig, ax = plt.subplots(2,3,sharex='all', sharey='all')

pred_tst_wfmean = wfmean.eval({X:Xtest})

names = ["Cd","Ni","Zn"]
for i in range(3):
    cx1 = ax[0,i].scatter(Xtest[:,0],Xtest[:,1],c=Ytest[:,i])
    cbar1 = fig.colorbar(cx1,ax=ax[0,i],fraction=0.046, pad=0.04)
    ax[0,i].set_title("actutal - "+ names[i])
    ax[0,i].xaxis.set_ticks([])
    ax[0,i].yaxis.set_ticks([])

    cx2 = ax[1,i].scatter(Xtest[:,0],Xtest[:,1],c=pred_tst_wfmean[:,i])
    cbar2 = fig.colorbar(cx2,ax=ax[1,i],fraction=0.046, pad=0.04)
    ax[1,i].set_title("predicted - "+ names[i]+"  noise var:"+str(np.round(noisevar_list[i].get_tfv().eval(),3)))
    ax[1,i].xaxis.set_ticks([])
    ax[1,i].yaxis.set_ticks([])

fig.tight_layout()
plt.show()